In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [29]:
import os
from __future__ import division, print_function
HOMEPATH = "/home/ubuntu/fastai/"
print("HOMEPATH:", HOMEPATH)
DATA_PATH = '/home/ubuntu/fastai/data/text_prediction/'
print("DATA_PATH:", DATA_PATH)


HOMEPATH: /home/ubuntu/fastai/
DATA_PATH: /home/ubuntu/fastai/data/text_prediction/


In [3]:
os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())

%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

current working directory: /home/ubuntu/fastai


Using Theano backend.


In [4]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

#https://keras.io/layers/wrappers/
#https://keras.io/layers/recurrent/#simplernn
#https://keras.io/activations/


In [5]:
input_file = 'pride_prejudice_trimmed.txt'
text = open(DATA_PATH + input_file).read().lower()
print('corpus length:', len(text))

corpus length: 705993


In [6]:
text

'\r\n\r\nit is a truth universally acknowledged, that a single man in possession\r\nof a good fortune, must be in want of a wife.\r\n\r\nhowever little known the feelings or views of such a man may be on his\r\nfirst entering a neighbourhood, this truth is so well fixed in the minds\r\nof the surrounding families, that he is considered the rightful property\r\nof some one or other of their daughters.\r\n\r\n\xe2\x80\x9cmy dear mr. bennet,\xe2\x80\x9d said his lady to him one day, \xe2\x80\x9chave you heard that\r\nnetherfield park is let at last?\xe2\x80\x9d\r\n\r\nmr. bennet replied that he had not.\r\n\r\n\xe2\x80\x9cbut it is,\xe2\x80\x9d returned she; \xe2\x80\x9cfor mrs. long has just been here, and she\r\ntold me all about it.\xe2\x80\x9d\r\n\r\nmr. bennet made no answer.\r\n\r\n\xe2\x80\x9cdo you not want to know who has taken it?\xe2\x80\x9d cried his wife impatiently.\r\n\r\n\xe2\x80\x9c_you_ want to tell me, and i have no objection to hearing it.\xe2\x80\x9d\r\n\r\nthis was i

In [7]:
print (type(text), len(text))
#chapter_remove = "Chapter "
chapter_remove = "\nchapter "
print (text.find(chapter_remove))

pos = 0
while pos != -1: # and (pos >=0 and pos<len(text)):
    pos = text.find(chapter_remove)
    pos_end = pos + text[pos+1:].find("\n")
    print ("pos:", pos)
    print ("pos_end:", pos_end)
    if pos != -1:
        print ("deleting text from text: '", text[pos:pos_end], "', len(text):", len(text))
        text = text.replace(text[pos:pos_end],"")
    
    #break
print ("end")

<type 'str'> 705993
238791
pos: 238791
pos_end: 238802
deleting text from text: ' 
chapter 24 ', len(text): 705993
pos: 249988
pos_end: 249999
deleting text from text: ' 
chapter 25 ', len(text): 705982
pos: 258809
pos_end: 258820
deleting text from text: ' 
chapter 26 ', len(text): 705971
pos: 271908
pos_end: 271919
deleting text from text: ' 
chapter 27 ', len(text): 705960
pos: 279363
pos_end: 279374
deleting text from text: ' 
chapter 28 ', len(text): 705949
pos: 287819
pos_end: 287830
deleting text from text: ' 
chapter 29 ', len(text): 705938
pos: 301960
pos_end: 301971
deleting text from text: ' 
chapter 30 ', len(text): 705927
pos: 309165
pos_end: 309176
deleting text from text: ' 
chapter 31 ', len(text): 705916
pos: 318145
pos_end: 318156
deleting text from text: ' 
chapter 32 ', len(text): 705905
pos: 326900
pos_end: 326911
deleting text from text: ' 
chapter 33 ', len(text): 705894
pos: 337649
pos_end: 337660
deleting text from text: ' 
chapter 34 ', len(text): 705883
pos: 

In [8]:
#check no more instances of text to remove remain
print (text.find(chapter_remove))


-1


In [13]:
print('corpus length:', len(text))
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)
print ("chars:", chars)

corpus length: 705575
total chars: 52
chars: ['\n', '\r', ' ', '!', '(', ')', '*', ',', '-', '.', '1', '2', '5', '6', '8', ':', ';', '?', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x98', '\x99', '\x9c', '\x9d', '\xe2']


In [14]:
chars.insert(0, "\0")

In [15]:
print ("chars:", chars)

chars: ['\x00', '\n', '\r', ' ', '!', '(', ')', '*', ',', '-', '.', '1', '2', '5', '6', '8', ':', ';', '?', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x98', '\x99', '\x9c', '\x9d', '\xe2']


In [16]:
''.join(chars[1:-6])
#display all the chars except last 6.

'\n\r !()*,-.12568:;?_abcdefghijklmnopqrstuvwxyz'

In [17]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [18]:
idx = [char_indices[c] for c in text]

In [19]:
idx[:10]

[2, 1, 2, 1, 28, 39, 3, 28, 38, 3]

In [20]:
''.join(indices_char[i] for i in idx[:80])

'\r\n\r\nit is a truth universally acknowledged, that a single man in possession\r\nof '

In [21]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 705536


In [22]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [23]:
sentences.shape, next_chars.shape

((705534, 40), (705534, 40))

In [24]:
n_fac = 24

In [65]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [66]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [67]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    print ("start, len(seed_string):", len(seed_string))
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
        print (i, ":", seed_string)
    print(seed_string)

In [68]:
def print_example_(seed_string, len_reqd):
    seed_string = seed_string.lower()
    print ("start, len(seed_string):", len(seed_string))
    print ("seed_string: '", seed_string, "'" )
    adequate_len = False
    if len(seed_string)>=len_reqd:
        adequate_len = True
        for i in range(len_reqd):
            x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
            preds = model.predict(x, verbose=0)[0][-1]
            preds = preds/np.sum(preds)
            next_char = choice(chars, p=preds)
            seed_string = seed_string + next_char
    return (adequate_len, seed_string)

In [69]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
705534/705534 [==============================] - 1738s - loss: 1.3340  


In [60]:
print_example()
#

start, len(seed_string): 40
0 : ethics is a basic foundation of all that 
1 : ethics is a basic foundation of all that s
2 : ethics is a basic foundation of all that sh
3 : ethics is a basic foundation of all that she
4 : ethics is a basic foundation of all that she 
5 : ethics is a basic foundation of all that she d
6 : ethics is a basic foundation of all that she da
7 : ethics is a basic foundation of all that she dar
8 : ethics is a basic foundation of all that she dari
9 : ethics is a basic foundation of all that she darin
10 : ethics is a basic foundation of all that she daring
11 : ethics is a basic foundation of all that she daring 
12 : ethics is a basic foundation of all that she daring w
13 : ethics is a basic foundation of all that she daring wi
14 : ethics is a basic foundation of all that she daring wit
15 : ethics is a basic foundation of all that she daring with
16 : ethics is a basic foundation of all that she daring with 
17 : ethics is a basic foundation of all that s

91 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quit
92 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quita
93 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitat
94 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitati
95 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitatio
96 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation
97 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation 
98 : ethics is a bas

143 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you
144 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you 
145 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you s
146 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you sa
147 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw
148 : ethics is a basic foundation of all tha

183 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he
184 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he 
185 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he d
186 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he di
187 : ethics is a basic foundation of all that she daring with the well
the ne

219 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there w
220 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there wa
221 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was
222 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most ref

251 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had diffi
252 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had diffic
253 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had difficu
254 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i m

279 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had difficult
towards them; and he d
280 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had difficult
towards them; and he du
281 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had difficult
towards them; and he dur
282 : ethics is a basic foun

307 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had difficult
towards them; and he during her eagerness at heari
308 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had difficult
towards them; and he during her eagerness at hearin
309 : ethics is a basic foundation of all that she daring with the well
the next marriage of a young man. i must once for such trust quitation noon
of the pleasanter counting, and when you saw her most refreshing for his way, he disappointed
her feelings; there was all rest day. darcy had difficu

In [63]:
print_example_("is it me you are looking for, I have always been here.", 60)

start, len(seed_string): 54
seed_string: ' is it me you are looking for, i have always been here. '


(False, 'is it me you are looking for, i have always been here.')

In [64]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
  3776/705534 [..............................] - ETA: 1762s - loss: 1.1462

KeyboardInterrupt: 

In [ ]:
model.optimizer.lr=0.001

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

In [ ]:
print_example()

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

In [ ]:
print_example()